In [1]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [5]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Podcast_Name', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [6]:
podcast = {}
for i, n in enumerate(train['Podcast_Name'].unique()):
    podcast[n] = i

categorical_map.update(podcast)

In [7]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [8]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [9]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [10]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [11]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [12]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Podcast_Name                                            750000 non-null  float64
 1   Episode_Title                                           750000 non-null  float64
 2   Episode_Length_minutes                                  750000 non-null  float64
 3   Genre                                                   750000 non-null  float64
 4   Host_Popularity_percentage                              750000 non-null  float64
 5   Publication_Day                                         750000 non-null  float64
 6   Publication_Time                                        750000 non-null  float64
 7   Guest_Popularity_percentage                             750000 non-null  float64
 8   Number_of_Ads           

In [13]:
important_features_bygain = ['Episode_Length_minutes','Episode_Title Episode_Length_minutes', 'Episode_Title', 'Podcast_Name','Number_of_Ads',
'Episode_Length_minutes Host_Popularity_percentage', 'Podcast_Name Episode_Length_minutes', 'Host_Popularity_percentage']
idx = []
for i, col in enumerate(x.columns): 
    if col in important_features_bygain: idx.append(i)

In [14]:
x_n = x.iloc[idx]
test_n = test.iloc[idx]

In [15]:
for c in categories: 
    x_n[c] = x_n[c].astype('category')
    test_n[c] = test_n[c].astype('category')

C:\Users\evan\AppData\Local\Temp\ipykernel_27508\4176405769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n[c] = x_n[c].astype('category')
C:\Users\evan\AppData\Local\Temp\ipykernel_27508\4176405769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_n[c] = test_n[c].astype('category')


In [16]:
K = 5
kf = KFold(K, shuffle=True, random_state=55)

In [17]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [18]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist", min_child_weight=10, learning_rate=0.2, n_estimators=1000, importance_type='gain')
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)

[0]	validation_0-rmse:23.13178
[1]	validation_0-rmse:20.14815
[2]	validation_0-rmse:17.97806
[3]	validation_0-rmse:16.43334
[4]	validation_0-rmse:15.35988
[5]	validation_0-rmse:14.62626
[6]	validation_0-rmse:14.13481
[7]	validation_0-rmse:13.80671
[8]	validation_0-rmse:13.59018
[9]	validation_0-rmse:13.44615
[10]	validation_0-rmse:13.35274
[11]	validation_0-rmse:13.28891
[12]	validation_0-rmse:13.24809
[13]	validation_0-rmse:13.21838
[14]	validation_0-rmse:13.19776
[15]	validation_0-rmse:13.18112
[16]	validation_0-rmse:13.17152
[17]	validation_0-rmse:13.16342
[18]	validation_0-rmse:13.15538
[19]	validation_0-rmse:13.14960
[20]	validation_0-rmse:13.14541
[21]	validation_0-rmse:13.14043
[22]	validation_0-rmse:13.13580
[23]	validation_0-rmse:13.13196
[24]	validation_0-rmse:13.12862
[25]	validation_0-rmse:13.12600
[26]	validation_0-rmse:13.12376
[27]	validation_0-rmse:13.12155
[28]	validation_0-rmse:13.11929
[29]	validation_0-rmse:13.11789
[30]	validation_0-rmse:13.11676
[31]	validation_0-

c:\Users\evan\anaconda3\lib\site-packages\xgboost\core.py:729: UserWarning: [23:41:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[0]	validation_0-rmse:23.05597
[1]	validation_0-rmse:20.07823
[2]	validation_0-rmse:17.91143
[3]	validation_0-rmse:16.37382
[4]	validation_0-rmse:15.30439
[5]	validation_0-rmse:14.57615
[6]	validation_0-rmse:14.08757
[7]	validation_0-rmse:13.76361
[8]	validation_0-rmse:13.55096
[9]	validation_0-rmse:13.41022
[10]	validation_0-rmse:13.31884
[11]	validation_0-rmse:13.25781
[12]	validation_0-rmse:13.21760
[13]	validation_0-rmse:13.18980
[14]	validation_0-rmse:13.16974
[15]	validation_0-rmse:13.15617
[16]	validation_0-rmse:13.14430
[17]	validation_0-rmse:13.13548
[18]	validation_0-rmse:13.12768
[19]	validation_0-rmse:13.12190
[20]	validation_0-rmse:13.11916
[21]	validation_0-rmse:13.11650
[22]	validation_0-rmse:13.11281
[23]	validation_0-rmse:13.10821
[24]	validation_0-rmse:13.10627
[25]	validation_0-rmse:13.10427
[26]	validation_0-rmse:13.09937
[27]	validation_0-rmse:13.09646
[28]	validation_0-rmse:13.09439
[29]	validation_0-rmse:13.09181
[30]	validation_0-rmse:13.08924
[31]	validation_0-

In [19]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [20]:
pred.head()

,0,0,0,0,0
750000,53.145958,52.794716,52.770439,52.671822,52.034229
750001,17.795708,17.923090,17.332899,17.762634,17.542501
750002,49.924648,50.417778,48.754372,53.980320,50.300041
750003,75.924591,75.495178,64.964180,70.554047,79.453415
750004,47.779255,48.283665,48.829681,52.592911,49.552921


In [21]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000    52.683434
750001    17.671368
750002    50.675434
750003    73.278282
750004    49.407688
Name: Listening_Time_minutes, dtype: float32

In [ ]:
pred_sum.to_csv('submissions/submission_newparams2.csv', index_label='id')